In [223]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch.utils.data import random_split
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu


In [224]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt

plt.style.use('ggplot')




solarDataset = pl.read_csv("Data/flare.data1",new_columns=['modified Zurich class','largest spot size','spot distribution','activity',
                                                           'evolution','previous 24 hour flare activity','historically-complex','became complex on this pass',
                                                           'area','area of largest spot','common flares','moderate flares',
                                                           'severe flares'] ,has_header=False,separator=' ',skip_rows=1)

In [225]:
display(solarDataset)
tamanhoV1 = solarDataset.estimated_size()
print(f"Memoria usada: {tamanhoV1}")

modified Zurich class,largest spot size,spot distribution,activity,evolution,previous 24 hour flare activity,historically-complex,became complex on this pass,area,area of largest spot,common flares,moderate flares,severe flares
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""C""","""S""","""O""",1,2,1,1,2,1,2,0,0,0
"""D""","""S""","""O""",1,3,1,1,2,1,2,0,0,0
"""C""","""S""","""O""",1,3,1,1,2,1,1,0,0,0
"""D""","""S""","""O""",1,3,1,1,2,1,2,0,0,0
"""D""","""A""","""O""",1,3,1,1,2,1,2,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…
"""C""","""R""","""O""",1,2,1,2,2,1,2,0,0,0
"""D""","""R""","""O""",1,3,1,1,2,1,2,0,0,0
"""E""","""A""","""O""",1,3,1,1,2,1,2,0,0,0


Memoria usada: 26809


In [226]:
solarDatasetv2 = pl.read_csv("Data/flare.data1", dtypes={
            "column_1": pl.Categorical,
            "column_2": pl.Categorical,
            "column_3": pl.Categorical,
            "column_4": pl.Float32,
            "column_5": pl.Float32,
            "column_6": pl.Float32,
            "column_7": pl.Float32,
            "column_8": pl.Float32,
            "column_9": pl.Float32,
            "column_10": pl.Float32,
            "column_11": pl.Float32,
            "column_12": pl.Float32,
            "column_13": pl.Float32,
        },has_header=False,new_columns=['modified Zurich class','largest spot size','spot distribution','activity',
                                                           'evolution','previous 24 hour flare activity','historically-complex','became complex on this pass',
                                                           'area','area of largest spot','common flares','moderate flares',
                                                           'severe flares'],skip_rows=1,separator=' ')
display(solarDatasetv2)
tamanhoV2 = solarDatasetv2.estimated_size()

print(f"Memoria usada: {tamanhoV2}, porcentaxe: {tamanhoV2/tamanhoV1 *100:2.2f}%")

modified Zurich class,largest spot size,spot distribution,activity,evolution,previous 24 hour flare activity,historically-complex,became complex on this pass,area,area of largest spot,common flares,moderate flares,severe flares
cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""C""","""S""","""O""",1.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0
"""D""","""S""","""O""",1.0,3.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0
"""C""","""S""","""O""",1.0,3.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0
"""D""","""S""","""O""",1.0,3.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0
"""D""","""A""","""O""",1.0,3.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…
"""C""","""R""","""O""",1.0,2.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0
"""D""","""R""","""O""",1.0,3.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0
"""E""","""A""","""O""",1.0,3.0,1.0,1.0,2.0,1.0,2.0,0.0,0.0,0.0


Memoria usada: 16980, porcentaxe: 63.34%


## Visualización

In [227]:
to_select = ['activity','evolution','previous 24 hour flare activity',	'historically-complex','became complex on this pass','area','area of largest spot']
medias = solarDatasetv2[to_select].mean()

grupo_numericos = solarDatasetv2.select(to_select)
display(grupo_numericos)
medias_solo = torch.tensor(grupo_numericos.mean().to_numpy()).squeeze()
stds = solarDatasetv2[to_select].std()
stds_solo = torch.tensor(grupo_numericos.std().to_numpy()).squeeze()
display(medias)
display(medias_solo)
display(medias_solo.shape)
display(medias_solo.shape)
display(stds)
display(stds_solo)


activity,evolution,previous 24 hour flare activity,historically-complex,became complex on this pass,area,area of largest spot
f32,f32,f32,f32,f32,f32,f32
1.0,2.0,1.0,1.0,2.0,1.0,2.0
1.0,3.0,1.0,1.0,2.0,1.0,2.0
1.0,3.0,1.0,1.0,2.0,1.0,1.0
1.0,3.0,1.0,1.0,2.0,1.0,2.0
1.0,3.0,1.0,1.0,2.0,1.0,2.0
…,…,…,…,…,…,…
1.0,2.0,1.0,2.0,2.0,1.0,2.0
1.0,3.0,1.0,1.0,2.0,1.0,2.0
1.0,3.0,1.0,1.0,2.0,1.0,2.0


activity,evolution,previous 24 hour flare activity,historically-complex,became complex on this pass,area,area of largest spot
f32,f32,f32,f32,f32,f32,f32
1.139319,2.486068,1.19195,1.368421,1.947368,1.027864,1.755418


tensor([1.1393, 2.4861, 1.1920, 1.3684, 1.9474, 1.0279, 1.7554])

torch.Size([7])

torch.Size([7])

activity,evolution,previous 24 hour flare activity,historically-complex,became complex on this pass,area,area of largest spot
f32,f32,f32,f32,f32,f32,f32
0.346816,0.601983,0.590029,0.483125,0.223643,0.164838,0.430506


tensor([0.3468, 0.6020, 0.5900, 0.4831, 0.2236, 0.1648, 0.4305])

## Scaler

In [228]:
class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native functions. The module does not expect the
        tensors to be of any specific shape; as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)
        

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)

    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)

    def __call__(self, sample):
        values,saidas = sample
        return ((values - self.mean) / (self.std + self.epsilon), saidas)
        
    def __repr__(self):
        return f"mean: {self.mean}, std:{self.std}, epsilon:{self.epsilon}"

# torch_tensor = torch.tensor(XSinHead.values)
# scaler = StandardScaler()
# scaler.fit(torch_tensor)
# display(scaler)
# XScalada = scaler.fit_transform(torch_tensor)

#### Instancia del estandar scaler

In [229]:
scaler = StandardScaler(medias_solo, stds_solo)
display(scaler)

mean: tensor([1.1393, 2.4861, 1.1920, 1.3684, 1.9474, 1.0279, 1.7554]), std:tensor([0.3468, 0.6020, 0.5900, 0.4831, 0.2236, 0.1648, 0.4305]), epsilon:1e-07

In [230]:
npSample = solarDatasetv2.select(to_select)[0].to_numpy()
tSample = torch.Tensor(npSample)
display(tSample)
sample_escalado = scaler.transform(tSample)
display(sample_escalado)

tensor([[1., 2., 1., 1., 2., 1., 2.]])

tensor([[-0.4017, -0.8074, -0.3253, -0.7626,  0.2353, -0.1690,  0.5681]])

## Get Dummies

In [231]:
dummies = solarDatasetv2.select([pl.col(pl.Categorical)])
print(dummies.columns)
dummies = [[ {"nome": columna, 'valor': i}
        for i in dummies.get_column(columna).cat.get_categories()
    ] for columna in dummies.columns]
dummies_flat = [item for row in dummies for item in row]
display(dummies_flat)
print(len(dummies_flat))

['modified Zurich class', 'largest spot size', 'spot distribution']


[{'nome': 'modified Zurich class', 'valor': 'C'},
 {'nome': 'modified Zurich class', 'valor': 'D'},
 {'nome': 'modified Zurich class', 'valor': 'B'},
 {'nome': 'modified Zurich class', 'valor': 'F'},
 {'nome': 'modified Zurich class', 'valor': 'H'},
 {'nome': 'modified Zurich class', 'valor': 'E'},
 {'nome': 'largest spot size', 'valor': 'S'},
 {'nome': 'largest spot size', 'valor': 'A'},
 {'nome': 'largest spot size', 'valor': 'K'},
 {'nome': 'largest spot size', 'valor': 'R'},
 {'nome': 'largest spot size', 'valor': 'X'},
 {'nome': 'largest spot size', 'valor': 'H'},
 {'nome': 'spot distribution', 'valor': 'O'},
 {'nome': 'spot distribution', 'valor': 'I'},
 {'nome': 'spot distribution', 'valor': 'X'},
 {'nome': 'spot distribution', 'valor': 'C'}]

16


In [232]:
novo_expr = [(pl.col(item['nome']) == item["valor"] ).alias(f'{item["nome"]}-{item["valor"]}') for item in dummies_flat]
print(novo_expr)

[<Expr ['[(col("modified Zurich class")…'] at 0x7FE45C119240>, <Expr ['[(col("modified Zurich class")…'] at 0x7FE45C11BFD0>, <Expr ['[(col("modified Zurich class")…'] at 0x7FE45C11A8F0>, <Expr ['[(col("modified Zurich class")…'] at 0x7FE45C119D50>, <Expr ['[(col("modified Zurich class")…'] at 0x7FE45C11B8B0>, <Expr ['[(col("modified Zurich class")…'] at 0x7FE45C119120>, <Expr ['[(col("largest spot size")) ==…'] at 0x7FE45C118490>, <Expr ['[(col("largest spot size")) ==…'] at 0x7FE45C11AC50>, <Expr ['[(col("largest spot size")) ==…'] at 0x7FE45C11B460>, <Expr ['[(col("largest spot size")) ==…'] at 0x7FE45C11B4C0>, <Expr ['[(col("largest spot size")) ==…'] at 0x7FE45C1195D0>, <Expr ['[(col("largest spot size")) ==…'] at 0x7FE45C11A140>, <Expr ['[(col("spot distribution")) ==…'] at 0x7FE45C118AC0>, <Expr ['[(col("spot distribution")) ==…'] at 0x7FE45C11B220>, <Expr ['[(col("spot distribution")) ==…'] at 0x7FE45C119420>, <Expr ['[(col("spot distribution")) ==…'] at 0x7FE45C118730>]


## Dataloader

In [233]:
class SolarDataset(Dataset):
  def __init__(self, src_file, root_dir, transform=None, expr_dummies = None):
    # data like: 5.0, 3.5, 1.3, 0.3, 0
    self.transform = transform
    self.expr_dummies = expr_dummies
    self.dataSet = pl.scan_csv(src_file, dtypes={
            "column_1": pl.Categorical,
            "column_2": pl.Categorical,
            "column_3": pl.Categorical,
            "column_4": pl.Float32,
            "column_5": pl.Float32,
            "column_6": pl.Float32,
            "column_7": pl.Float32,
            "column_8": pl.Float32,
            "column_9": pl.Float32,
            "column_10": pl.Float32,
            "column_11": pl.Float32,
            "column_12": pl.Float32,
            "column_13": pl.Float32
        },has_header=False,skip_rows=1,separator=' ',null_values=['?']).drop_nulls().rename({
            "column_1": "modified Zurich class",
            "column_2": "largest spot size",
            "column_3": "spot distribution",
            "column_4": "activity",
            "column_5": "evolution",
            "column_6": "previous 24 hour flare activity",
            "column_7": "historically-complex",
            "column_8": "became complex on this pass",
            "column_9": "area",
            "column_10": "area of largest spot",
            "column_11": "common flares",
            "column_12": "moderate flares",
            "column_13": "severe flares"
        }).with_row_index("id")
    

  def __len__(self):
    return self.dataSet.select(pl.len()).collect().item()

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    else:
      idx = [idx]
    # en datos vou a ter as filas que se pediron a __getitem__
    seccion = self.dataSet.filter(pl.col("id").is_in(idx)).drop("id")
    datos = seccion.collect()
    input_cols = [
            "activity", "evolution", "previous 24 hour flare activity", 
            "historically-complex", "became complex on this pass", 
            "area", "area of largest spot","modified Zurich class",
            "largest spot size","spot distribution"
        ]
    # en datosNumericos teño as columnas con valores numericos, se van a escalar    
    datosEntrada = datos.select(input_cols)
    datosNumericos = datosEntrada.select([pl.col(pl.Int8),pl.col(pl.Float32)])
    print(datosNumericos.to_numpy())
    predsA =  self.transform.transform(torch.tensor(datosNumericos.to_numpy()).squeeze()) 
    print(predsA)
    # isto ten 15 elementos scalados. os 14 primeiros son datos de entrada, o 15 e a saída desexada

    # en predsC vou a ter as columnas feitas dummies
    predsC = datos.select([pl.col(pl.Categorical)]).with_columns(
      self.expr_dummies
    ).drop(datos.select([pl.col(pl.Categorical)]).columns)

    print(predsC)
    # se transforma o dataset cos dummies en un tensor
    tensorB = torch.tensor(predsC.to_numpy().astype(np.int32)).squeeze()
    
    # dim=-1 para que se poñan un tensor ao lado do outro, senon se apilarían en vertical e non coinciden os tamaños
    entrada = torch.cat((predsA[:15], tensorB),dim=-1)
    # as veces pode ser que predsA sexa un vector dunha dimensión, cando o parametro idx é un numero
    # ou pode ser que sexa un unha matriz, se idx é un vector (se piden varios samples para batches)
    # en todo caso se quere o elemento 16, das filas que sexan
    output_cols = ["common flares", "moderate flares", "severe flares"]
    target = datos.select(output_cols).to_numpy().astype(np.float32)
    target = torch.tensor(target).squeeze()
    return   entrada, target

       

dataset = SolarDataset("Data/flare.data1",".",transform=scaler,expr_dummies=novo_expr)
print(dataset[0])
print(dataset[10])
print(dataset[torch.tensor([0,10,100])])

[[1. 2. 1. 1. 2. 1. 2.]]
tensor([-0.4017, -0.8074, -0.3253, -0.7626,  0.2353, -0.1690,  0.5681])
shape: (1, 16)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ modified  ┆ modified  ┆ modified  ┆ modified  ┆ … ┆ spot dist ┆ spot dist ┆ spot dist ┆ spot dis │
│ Zurich    ┆ Zurich    ┆ Zurich    ┆ Zurich    ┆   ┆ ribution- ┆ ribution- ┆ ribution- ┆ tributio │
│ class-C   ┆ class-D   ┆ class-B   ┆ class-F   ┆   ┆ O         ┆ I         ┆ X         ┆ n-C      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ bool      ┆ bool      ┆ bool      ┆ bool      ┆   ┆ bool      ┆ bool      ┆ bool      ┆ bool     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ true      ┆ false     ┆ false     ┆ false     ┆ … ┆ true      ┆ false     ┆ false     ┆ false    │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───

## Division Train-test

In [234]:
lonxitudeDataset = len(dataset)
print(f"Tamanho dataset {lonxitudeDataset}")
tamTrain =int(lonxitudeDataset*0.8)
tamVal = lonxitudeDataset - tamTrain
print(f"Tam dataset: {lonxitudeDataset} train: {tamTrain} tamVal: {tamVal}")
train_set, val_set = random_split(dataset,[tamTrain,tamVal])
train_ldr = torch.utils.data.DataLoader(train_set, batch_size=2,
    shuffle=True, drop_last=False)
validation_loader =torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False)

Tamanho dataset 323
Tam dataset: 323 train: 258 tamVal: 65


## Creación NN

In [235]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x


model     = Model(23) # Siempre Poner tamaño de la matriz de entrada
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.MSELoss()
compiled_model = torch.compile(model)
model
compiled_model

OptimizedModule(
  (_orig_mod): Model(
    (layer1): Linear(in_features=23, out_features=50, bias=True)
    (layer2): Linear(in_features=50, out_features=50, bias=True)
    (layer3): Linear(in_features=50, out_features=3, bias=True)
  )
)

In [236]:
entradaProba,dest = next(iter(train_ldr))
print('entrada')
print(entradaProba.dtype)
print(model.layer1.weight.dtype)
print('entrada proba')
display(entradaProba)
saida = model(entradaProba)
display(dest)
print('saida')
display(saida)
print('loss')
display(loss_fn(saida, dest ))

[[2. 1. 1. 1. 2. 1. 1.]]
tensor([ 2.4817, -2.4686, -0.3253, -0.7626,  0.2353, -0.1690, -1.7547])
shape: (1, 16)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ modified  ┆ modified  ┆ modified  ┆ modified  ┆ … ┆ spot dist ┆ spot dist ┆ spot dist ┆ spot dis │
│ Zurich    ┆ Zurich    ┆ Zurich    ┆ Zurich    ┆   ┆ ribution- ┆ ribution- ┆ ribution- ┆ tributio │
│ class-C   ┆ class-D   ┆ class-B   ┆ class-F   ┆   ┆ O         ┆ I         ┆ X         ┆ n-C      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ bool      ┆ bool      ┆ bool      ┆ bool      ┆   ┆ bool      ┆ bool      ┆ bool      ┆ bool     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ false     ┆ false     ┆ true      ┆ false     ┆ … ┆ true      ┆ false     ┆ false     ┆ false    │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───

tensor([[ 2.4817, -2.4686, -0.3253, -0.7626,  0.2353, -0.1690, -1.7547,  0.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000],
        [ 2.4817,  0.8537, -0.3253,  1.3073,  0.2353, -0.1690,  0.5681,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000]])

tensor([[1., 0., 0.],
        [0., 0., 0.]])

saida


tensor([[-0.0095,  0.0624, -0.0127],
        [ 0.0488,  0.0586,  0.0491]], grad_fn=<AddmmBackward0>)

loss


tensor(0.1719, grad_fn=<MseLossBackward0>)

## Metrics

In [237]:
from torchmetrics import MeanSquaredError, MeanAbsoluteError, R2Score
mean_squared_error = MeanSquaredError()
mean_absolute_error = MeanAbsoluteError()
r2Score = R2Score()
model.eval()
with torch.no_grad():
    for entradas, saidas in validation_loader:
        entradas, saidas = entradas.to(device), saidas.to(device)
        # Predicciones
        voutputs = model(entradas)
        # Aplanar para calcular los errores
        voutputs = voutputs.view(-1)
        saidas = saidas.view(-1)
        # Update metrics
        mean_squared_error.update(voutputs, saidas)
        mean_absolute_error.update(voutputs, saidas)
        r2Score.update(voutputs, saidas)
errorMedio = mean_squared_error.compute()
errorAbsolute =mean_absolute_error.compute()
r2 = r2Score.compute()
print(f"MSE: {errorMedio}")
print(f"MAE: {errorAbsolute}")
print(f"R^2: {r2}")


[[1. 2. 1. 1. 2. 1. 2.]]
tensor([-0.4017, -0.8074, -0.3253, -0.7626,  0.2353, -0.1690,  0.5681])
shape: (1, 16)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ modified  ┆ modified  ┆ modified  ┆ modified  ┆ … ┆ spot dist ┆ spot dist ┆ spot dist ┆ spot dis │
│ Zurich    ┆ Zurich    ┆ Zurich    ┆ Zurich    ┆   ┆ ribution- ┆ ribution- ┆ ribution- ┆ tributio │
│ class-C   ┆ class-D   ┆ class-B   ┆ class-F   ┆   ┆ O         ┆ I         ┆ X         ┆ n-C      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ bool      ┆ bool      ┆ bool      ┆ bool      ┆   ┆ bool      ┆ bool      ┆ bool      ┆ bool     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ false     ┆ false     ┆ false     ┆ false     ┆ … ┆ false     ┆ false     ┆ true      ┆ false    │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───

tensor([-0.4017,  0.8537, -0.3253, -0.7626,  0.2353, -0.1690,  0.5681])
shape: (1, 16)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ modified  ┆ modified  ┆ modified  ┆ modified  ┆ … ┆ spot dist ┆ spot dist ┆ spot dist ┆ spot dis │
│ Zurich    ┆ Zurich    ┆ Zurich    ┆ Zurich    ┆   ┆ ribution- ┆ ribution- ┆ ribution- ┆ tributio │
│ class-C   ┆ class-D   ┆ class-B   ┆ class-F   ┆   ┆ O         ┆ I         ┆ X         ┆ n-C      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ bool      ┆ bool      ┆ bool      ┆ bool      ┆   ┆ bool      ┆ bool      ┆ bool      ┆ bool     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ false     ┆ false     ┆ true      ┆ false     ┆ … ┆ true      ┆ false     ┆ false     ┆ false    │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘
[[1.